In [1]:
import pydoop.hdfs as hdfs

In [2]:
hdfs.lsl('hdfs://hdfs:9000/')

[]

In [3]:
hdfs.mkdir('hdfs://hdfs:9000/user/jupyter', user='root')

In [4]:
hdfs.chown('hdfs://hdfs:9000/user/jupyter', 'jupyter', hdfs_user='root')

In [5]:
hdfs.mkdir('hdfs://hdfs:9000/user/jupyter/tiledb_arrays')

In [6]:
for x in hdfs.lsl('hdfs://hdfs:9000/user/jupyter/tiledb_arrays'):
    hdfs.rmr(x['name'])

In [21]:
hdfs.put('../notebooks/data/sensor_data_denorm.csv', 'hdfs://hdfs:9000/user/jupyter/tiledb_arrays/junk2.csv')

This example is based on the [quickstart_dense](https://github.com/TileDB-Inc/TileDB-Py/blob/dev/examples/quickstart_dense.py) example.

In [8]:
import numpy as np
import tiledb

In [9]:
a_name = 'qdense'

In [10]:
ctx = tiledb.Ctx({'vfs.hdfs.username': 'jupyter', 'vfs.hdfs.name_node':'hdfs://hdfs:9000', 
                  'vfs.hdfs.name_node_uri':'hdfs://hdfs:9000'})

In [11]:
vfs = tiledb.VFS(ctx)

In [12]:
vfs.supports('hdfs')

True

In [18]:
vfs.is_dir('hdfs://hdfs:9000/user/jupyter/tiledb_arrays')

True

In [19]:
dom = tiledb.Domain(ctx, 
          tiledb.Dim(ctx, name="rows", domain=(1, 4), tile=4, dtype=np.int32),
          tiledb.Dim(ctx, name="cols", domain=(1, 4), tile=4, dtype=np.int32),
          )

In [20]:
schema = tiledb.ArraySchema(ctx, domain=dom, sparse=False,
                            attrs=[tiledb.Attr(ctx, name="a", dtype=np.int32)])

In [22]:
a_name = 'hdfs://hdfs:9000/user/jupyter/tiledb_arrays/'
tiledb.DenseArray.create(a_name, schema)

TileDBError: [TileDB::HDFS] Error: Cannot write to file hdfs://hdfs:9000/user/jupyter/tiledb_arrays/__array_schema.tdb; File closing error

In [ ]:
tiledb.libtiledb.version()  

In [ ]:


def write_array():
    ctx = tiledb.Ctx()
    # Open the array and write to it.
    with tiledb.DenseArray(ctx, array_name, mode='w') as A:
        data = np.array(([1, 2, 3, 4],
                         [5, 6, 7, 8],
                         [9, 10, 11, 12],
                         [13, 14, 15, 16]))
        A[:] = data


def read_array():
    ctx = tiledb.Ctx()
    # Open the array and read from it.
    with tiledb.DenseArray(ctx, array_name, mode='r') as A:
        # Slice only rows 1, 2 and cols 2, 3, 4.
        data = A[1:3, 2:5]
        print(data["a"])


ctx = tiledb.Ctx()
if tiledb.object_type(ctx, array_name) != "array":
    create_array()
    write_array()

read_array()
